In [1]:
import tensorflow as tf
gpu = tf.config.experimental.list_physical_devices('GPU')[0]
if len(gpu) > 0:
    tf.config.experimental.set_memory_growth(gpu, True)
from src.model.data_loader import DataLoader
import json
import os

In [2]:
import mlflow
import mlflow.tensorflow
from mlflow.tracking import MlflowClient

In [3]:
BATCH_SIZE = 16

data_loader = DataLoader(shape=(640,640), batch_size = BATCH_SIZE) 
data_loader.load_from_directory("/home/mugesh/IB/DocuNet/data/processed_data/")
data_gen = data_loader.data_generator(shape=(640,640), batch_size= BATCH_SIZE)

Loaded 696 files of 4 classes


In [4]:
## Model Architecture
input_ = tf.keras.layers.Input(shape=(640,640,1), name ="input")

conv_1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = "same", name="conv_1")(input_)
act_1 = tf.keras.layers.Activation('relu', name='act_1')(conv_1)
pool_1 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_1")(act_1)

conv_2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = "same" , name="conv_2")(pool_1)
act_2 = tf.keras.layers.Activation('relu', name='act_2')(conv_2)
pool_2 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_2")(act_2)

conv_3 = tf.keras.layers.Conv2D(filters = 16, kernel_size = (3,3), padding = "same",  name="conv_3")(pool_2)
act_3 = tf.keras.layers.Activation('relu', name='act_3')(conv_3)
pool_3 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_3")(act_3)

conv_4 = tf.keras.layers.Conv2D(filters = 16, kernel_size = (3,3), padding = "same", name="conv_4")(pool_3)
act_4 = tf.keras.layers.Activation('relu', name='act_4')(conv_4)
pool_4 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_4")(act_4)
pool_5 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_5")(pool_4)

flatten = tf.keras.layers.Flatten(name="flatten")(pool_5)


dense_1 = tf.keras.layers.Dense(64, activation='relu', name = "dense_1")(flatten)
dense_2 = tf.keras.layers.Dense(32, activation='relu', name = "dense_2")(dense_1)

shaded = tf.keras.layers.Dense(1,activation='sigmoid', name="shaded")(dense_2)
gridline = tf.keras.layers.Dense(1,activation='sigmoid', name="gridline")(dense_2)
good = tf.keras.layers.Dense(1,activation='sigmoid', name="good")(dense_2)
black_border = tf.keras.layers.Dense(1,activation='sigmoid', name="black_border")(dense_2)

model_outputs = {'black_border' : black_border, 'good' : good, 'gridline' : gridline, 'shaded' : shaded}


model = tf.keras.models.Model(input_, model_outputs, name ="DockNet")
model.summary()

## Compile 
model.compile(
    loss={
        "shaded": 'binary_crossentropy',
        "gridline": "binary_crossentropy",
        "good": "binary_crossentropy",
        "black_border": "binary_crossentropy"
    },
    optimizer='adam',
    metrics=['accuracy']
)

Model: "DockNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 640, 640, 1) 0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 640, 640, 32) 320         input[0][0]                      
__________________________________________________________________________________________________
act_1 (Activation)              (None, 640, 640, 32) 0           conv_1[0][0]                     
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 320, 320, 32) 0           act_1[0][0]                      
____________________________________________________________________________________________

In [5]:
experiment_name = "DockNet"

In [6]:
mlflow.tensorflow.autolog()
with mlflow.start_run(run_name=experiment_name):
    history = model.fit(
        data_gen,
        steps_per_epoch= data_loader.num_files // BATCH_SIZE,
        batch_size = BATCH_SIZE,
        epochs=5)

Epoch 1/5
43/43 [==============================] - 15s 248ms/step - loss: 1.4835 - black_border_loss: 0.4462 - good_loss: 0.3296 - gridline_loss: 0.2877 - shaded_loss: 0.4201 - black_border_accuracy: 0.7693 - good_accuracy: 0.8470 - gridline_accuracy: 0.8532 - shaded_accuracy: 0.7854
Epoch 2/5
43/43 [==============================] - 11s 246ms/step - loss: 0.1488 - black_border_loss: 0.0159 - good_loss: 0.0032 - gridline_loss: 0.0665 - shaded_loss: 0.0631 - black_border_accuracy: 0.9935 - good_accuracy: 0.9995 - gridline_accuracy: 0.9874 - shaded_accuracy: 0.9913
Epoch 3/5
43/43 [==============================] - 10s 245ms/step - loss: 0.0286 - black_border_loss: 0.0037 - good_loss: 0.0062 - gridline_loss: 0.0128 - shaded_loss: 0.0058 - black_border_accuracy: 1.0000 - good_accuracy: 1.0000 - gridline_accuracy: 0.9954 - shaded_accuracy: 0.9975
Epoch 4/5
43/43 [==============================] - 11s 246ms/step - loss: 8.9946e-04 - black_border_loss: 2.8708e-04 - good_loss: 2.0886e-04 - gr

2022/03/07 12:37:19 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp9bmrv5qv/model, flavor: keras), fall back to return ['tensorflow==2.4.2']. Set logging level to DEBUG to see the full traceback.
